# Part-1 : Fetch the reviews of the product provided by the user

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Where the Main and sub Review are present

In [5]:
def fetch_reviews_multiple(url, start=1, end=20):
    reviews_list = []

    for i in range(start, end + 1):
        url_page = f"{url}&page={i}"
        response = requests.get(url_page)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        reviews_container = soup.find_all("div", class_="cPHDOP col-12-12")
        if not reviews_container:
            continue  # Skip pages with no reviews

        review_divs = soup.find_all("div", class_="ZmyHeo")
        for div in review_divs:
            review_text = div.div.text.strip()
            review_text = review_text.replace("READ MORE","")
            reviews_list.append(review_text)
            
    return pd.DataFrame({'review': reviews_list})

# Where the Single Review is present

In [8]:
def fetch_reviews_single(url, start=1, end=20):
    reviews_list = []

    for i in range(start, end + 1):
        url_page = f"{url}&page={i}"
        response = requests.get(url_page)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        reviews_container = soup.find_all("div", class_="ZmyHeo MDcJkH")
        if not reviews_container:
            continue  # Skip pages with no reviews

        review_divs = soup.find_all("div", class_="_11pzQk")
        for div in review_divs:
            # review_text = div.div.text.strip()
            review_text = div.get_text(strip=True)
            review_text = review_text.replace("READ MORE","")
            reviews_list.append(review_text)
            
    return pd.DataFrame({'review': reviews_list})

# Final function

In [11]:
def Fetch_Review(url,Type='multiple'):
    df_final = pd.DataFrame()
    if Type == 'single':
            df1 = fetch_reviews_single(url)
            df_final = df1
    elif Type == 'multiple':
            df2 = fetch_reviews_multiple(url)
            df_final = df2
    else:
        print('wrong type')
    return df_final

# user input which show in frontend

In [69]:
url = input("Enter link :")
dff = Fetch_Review(url,Type='single')

Enter link : https://www.flipkart.com/imsa-moda-lingerie-set/product-reviews/itmc76d7e675de15?pid=LINGNY8ZJUBE58SN&lid=LSTLINGNY8ZJUBE58SNDBLQMD&marketplace=FLIPKART


In [71]:
dff

,review
0,Good 👌👌👌
1,Soft and comfortable
2,Very nice product 😃
3,I am very much shocked about this quality it i...
4,Product is very good fitting is perfect I love it
5,"Looks fabulous, my wife likes this product."
6,Good product within price
7,It's amazing 😍 I love it
8,"Very satisfying product, good quality same as ..."
9,Very good product 👍 nice fabric


# Part-2 : Check the sentiment of the reviews fetched in part 1 

In [20]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

# The Sentiment_model function is replaced my the actual traied LSTM model 

In [23]:
def sentiment_model(text):
    sia = SentimentIntensityAnalyzer()
    over_all_polarity = sia.polarity_scores(text)
    if over_all_polarity['compound'] >= 0.05:
        return "positive"
    elif over_all_polarity['compound'] <= -0.05:
        return "negative"
    else:
        return "neutral"

# This is the Function use in actual backend for check the polarity of the reviews

In [26]:
def Check_sentiment(df):
    df_with_polarity = df.copy()
    df_with_polarity['sentiment'] = df_with_polarity['review'].apply(sentiment_model)
    return df_with_polarity

In [73]:
final_df = Check_sentiment(dff)

In [75]:
final_df

,review,sentiment
0,Good 👌👌👌,positive
1,Soft and comfortable,positive
2,Very nice product 😃,positive
3,I am very much shocked about this quality it i...,positive
4,Product is very good fitting is perfect I love it,positive
5,"Looks fabulous, my wife likes this product.",positive
6,Good product within price,positive
7,It's amazing 😍 I love it,positive
8,"Very satisfying product, good quality same as ...",positive
9,Very good product 👍 nice fabric,positive


In [59]:
final_df['sentiment'].value_counts()

sentiment
positive    58
negative    16
neutral      6
Name: count, dtype: int64

# Final report for product

In [95]:
def get_review_text(final_df):
    positive_reviews = final_df[final_df['sentiment'] == 'positive']['review'].tolist()
    negative_reviews = final_df[final_df['sentiment'] == 'negative']['review'].tolist()
    neural_reviews = final_df[final_df['sentiment'] == 'neutral']['review'].tolist()
    return positive_reviews,negative_reviews,neural_reviews

In [97]:
pl,nel,neul = get_review_text(final_df)

In [99]:
print(pl,nel,neul)

['Good 👌👌👌', 'Soft and comfortable', 'Very nice product 😃', 'I am very much shocked about this quality it is very light and good quality product ❤️', 'Product is very good fitting is perfect I love it', 'Looks fabulous, my wife likes this product.', 'Good product within price', "It's amazing 😍 I love it", 'Very satisfying product, good quality same as the photo. Just go for it', 'Very good product 👍 nice fabric', 'Good', 'Good', 'Good 😊', 'Very nice product', 'Good 👍', 'Very good product', 'Good product', 'Good product', 'Good', 'Good quality', 'Good pr', 'Nice ❤️', 'Nice', 'Very sexy', 'Nice', 'Nice 👍', 'Product is nice but size is little bit big', 'Nice nice', 'Very good', 'Good work', 'Nice', 'Nice', 'Good', 'Nice', 'Nice but cotton not this is silky', 'Good fit', 'Good product', 'Best', 'Very nice product it is so comfortable', 'Good', 'Very good', 'Good luck', 'gud product value of money', 'Nice product', 'Ok ok', 'Good'] ['Panty is very bad cloth of panty is low quality Bra is ve

In [53]:
final_df[final_df['sentiment'] == 'negative'].shape[0]

16

In [55]:
final_df[final_df['sentiment'] == 'neutral'].shape[0]

6

In [83]:
def sentiment_report(df):
    final_report = df.copy()
    total = final_report['sentiment'].shape[0]
    
    positive = final_df[final_df['sentiment'] == 'positive'].shape[0]
    negative = final_df[final_df['sentiment'] == 'negative'].shape[0]
    neutral = final_df[final_df['sentiment'] == 'neutral'].shape[0]

    positive_presentage = (positive/total)*100
    negative_presentage = (negative/total)*100
    neutral_presentage = (neutral/total)*100
    return positive_presentage,negative_presentage,neutral_presentage

In [85]:
p,ne,nu = sentiment_report(final_df)

In [87]:
print(p,ne,nu)

76.66666666666667 15.0 8.333333333333332
